# Installations

In [ ]:
!git clone https://github.com/state-spaces/mamba.git
# !git clone https://github.com/xtwigs/mamba.git

Cloning into 'mamba'...
remote: Enumerating objects: 723, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 723 (delta 26), reused 15 (delta 15), pack-reused 684 (from 3)
Receiving objects: 100% (723/723), 1.58 MiB | 8.12 MiB/s, done.
Resolving deltas: 100% (386/386), done.


In [ ]:
%%time
# !pip install -q --cache-dir /kaggle/working/pip_cache/mamba_pkg /kaggle/working/mamba
!pip install -q /kaggle/working/mamba

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
CPU times: user 2.93 s, sys: 616 ms, total: 3.54 s
Wall time: 2min 46s


In [ ]:
%%time
# !pip install -q --cache-dir /kaggle/working/pip_cache/causal_conv causal-conv1d>=1.4.0
!pip install -q causal-conv1d>=1.4.0
# !pip install -q mamba-ssm
# !pip install -q mamba-ssm[causal-conv1d]
# !pip install -q mamba-ssm[dev]

CPU times: user 251 ms, sys: 72.8 ms, total: 323 ms
Wall time: 14.7 s


In [ ]:
%%time
!pip install triton

CPU times: user 66.8 ms, sys: 15.8 ms, total: 82.6 ms
Wall time: 3.48 s


In [ ]:
%%time
!pip -q install lightning torchscale evaluate huggingface_hub unbabel-comet flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 69.9 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101

In [ ]:
%%time
!pip install -q x-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.0 MB/s eta 0:00:00
CPU times: user 87.6 ms, sys: 26.4 ms, total: 114 ms
Wall time: 4.63 s


# Imports

In [ ]:
import numpy as np
import pandas as pd
import torch
import re
import matplotlib.pyplot as plt

# Data

In [ ]:
def read_txt(path):
    with open(path,'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    text_pairs = []
    mx = 0
    mn = 100
    x = []
    for _line in lines:
        arr = _line.split(':')
        if len(arr) == 1:
            continue
        text_pairs.append({'Event Type': arr[0], \
                          'Feynman Diagram': arr[1],   \
                           'Amplitude': arr[-2],        \
                           'Squared Amplitude': arr[-1] \
                          })
    return text_pairs

final_pairs = [read_txt(
            f'/kaggle/input/squared-amplitudes-test-data/SYMBA - Test Data/QED-2-to-2-diag-TreeLevel-{i}.txt')
            for i in range(10)]
final_pairs = [xx for x in final_pairs for xx in x]

# df = pd.DataFrame(final_pairs,columns=['Event Type','Feynman Diagram', 'Amplitude', 'Squared Amplitude'],dtype=['str','str','str','str'])
df = pd.DataFrame(final_pairs, columns=['Event Type', 'Feynman Diagram', 'Amplitude', 'Squared Amplitude'])
df = df.astype({'Event Type': 'string', 'Feynman Diagram': 'string', 'Amplitude': 'string', 'Squared Amplitude': 'string'})

from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, shuffle=True)

print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")

train_df.to_csv('train.csv')
val_df.to_csv('val.csv')
test_df.to_csv('test.csv')

Train size: 12441, Validation size: 1555, Test size: 1556


In [ ]:
# import numpy as np
# import pandas as pd

# # Define target column name - easy to change
# target_column = 'Squared Amplitude'  # Change this to your actual target column name

# # Get unique targets and shuffle them
# unique_targets = df[target_column].unique()
# np.random.seed(42)  # For reproducibility
# np.random.shuffle(unique_targets)

# # Calculate split points for 70/15/15 distribution
# n_targets = len(unique_targets)
# train_idx = int(0.7 * n_targets)
# val_idx = int(0.85 * n_targets)

# # Split targets into train/val/test groups
# train_targets = unique_targets[:train_idx]
# val_targets = unique_targets[train_idx:val_idx]
# test_targets = unique_targets[val_idx:]

# # Create the dataframes based on target groups
# train_df = df[df[target_column].isin(train_targets)].sample(frac=1, random_state=42).reset_index(drop=True)
# val_df = df[df[target_column].isin(val_targets)].sample(frac=1, random_state=42).reset_index(drop=True)
# test_df = df[df[target_column].isin(test_targets)].sample(frac=1, random_state=42).reset_index(drop=True)

# # Print statistics
# print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")
# print(f"Train targets: {len(train_targets)}, Val targets: {len(val_targets)}, Test targets: {len(test_targets)}")

# # Save the datasets
# train_df.to_csv('train.csv')
# val_df.to_csv('val.csv')
# test_df.to_csv('test.csv')

# Vocab

In [ ]:
def reporthook(t):
    """
    https://github.com/tqdm/tqdm.
    """
    last_b = [0]

    def inner(b=1, bsize=1, tsize=None):
        """
        b: int, optional
        Number of blocks just transferred [default: 1].
        bsize: int, optional
        Size of each block (in tqdm units) [default: 1].
        tsize: int, optional
        Total size (in tqdm units). If [default: None] remains unchanged.
        """
        if tsize is not None:
            t.total = tsize
        t.update((b - last_b[0]) * bsize)
        last_b[0] = b

    return inner

In [ ]:
import torch
import logging
import os
import zipfile
import gzip
from urllib.request import urlretrieve
from tqdm import tqdm
import tarfile
from functools import partial

logger = logging.getLogger(__name__)


def _infer_shape(f):
    num_lines, vector_dim = 0, None
    for line in f:
        if vector_dim is None:
            row = line.rstrip().split(b" ")
            vector = row[1:]
            # Assuming word, [vector] format
            if len(vector) > 2:
                # The header present in some (w2v) formats contains two elements.
                vector_dim = len(vector)
                num_lines += 1  # First element read
        else:
            num_lines += 1
    f.seek(0)
    return num_lines, vector_dim


class Vectors(object):

    def __init__(self, name, cache=None,
                 url=None, unk_init=None, max_vectors=None):
        """
        Args:

            name: name of the file that contains the vectors
            cache: directory for cached vectors
            url: url for download if vectors not found in cache
            unk_init (callback): by default, initialize out-of-vocabulary word vectors
                to zero vectors; can be any function that takes in a Tensor and returns a Tensor of the same size
            max_vectors (int): this can be used to limit the number of
                pre-trained vectors loaded.
                Most pre-trained vector sets are sorted
                in the descending order of word frequency.
                Thus, in situations where the entire set doesn't fit in memory,
                or is not needed for another reason, passing `max_vectors`
                can limit the size of the loaded set.
        """

        cache = '.vector_cache' if cache is None else cache
        self.itos = None
        self.stoi = None
        self.vectors = None
        self.dim = None
        self.unk_init = torch.Tensor.zero_ if unk_init is None else unk_init
        self.cache(name, cache, url=url, max_vectors=max_vectors)

    def __getitem__(self, token):
        if token in self.stoi:
            return self.vectors[self.stoi[token]]
        else:
            return self.unk_init(torch.Tensor(self.dim))

    def cache(self, name, cache, url=None, max_vectors=None):
        import ssl
        ssl._create_default_https_context = ssl._create_unverified_context
        if os.path.isfile(name):
            path = name
            if max_vectors:
                file_suffix = '_{}.pt'.format(max_vectors)
            else:
                file_suffix = '.pt'
            path_pt = os.path.join(cache, os.path.basename(name)) + file_suffix
        else:
            path = os.path.join(cache, name)
            if max_vectors:
                file_suffix = '_{}.pt'.format(max_vectors)
            else:
                file_suffix = '.pt'
            path_pt = path + file_suffix

        if not os.path.isfile(path_pt):
            if not os.path.isfile(path) and url:
                logger.info('Downloading vectors from {}'.format(url))
                if not os.path.exists(cache):
                    os.makedirs(cache)
                dest = os.path.join(cache, os.path.basename(url))
                if not os.path.isfile(dest):
                    with tqdm(unit='B', unit_scale=True, miniters=1, desc=dest) as t:
                        try:
                            urlretrieve(url, dest, reporthook=reporthook(t))
                        except KeyboardInterrupt as e:  # remove the partial zip file
                            os.remove(dest)
                            raise e
                logger.info('Extracting vectors into {}'.format(cache))
                ext = os.path.splitext(dest)[1][1:]
                if ext == 'zip':
                    with zipfile.ZipFile(dest, "r") as zf:
                        zf.extractall(cache)
                elif ext == 'gz':
                    if dest.endswith('.tar.gz'):
                        with tarfile.open(dest, 'r:gz') as tar:
                            tar.extractall(path=cache)
            if not os.path.isfile(path):
                raise RuntimeError('no vectors found at {}'.format(path))

            logger.info("Loading vectors from {}".format(path))
            ext = os.path.splitext(path)[1][1:]
            if ext == 'gz':
                open_file = gzip.open
            else:
                open_file = open

            vectors_loaded = 0
            with open_file(path, 'rb') as f:
                num_lines, dim = _infer_shape(f)
                if not max_vectors or max_vectors > num_lines:
                    max_vectors = num_lines

                itos, vectors, dim = [], torch.zeros((max_vectors, dim)), None

                for line in tqdm(f, total=max_vectors):
                    # Explicitly splitting on " " is important, so we don't
                    # get rid of Unicode non-breaking spaces in the vectors.
                    entries = line.rstrip().split(b" ")

                    word, entries = entries[0], entries[1:]
                    if dim is None and len(entries) > 1:
                        dim = len(entries)
                    elif len(entries) == 1:
                        logger.warning("Skipping token {} with 1-dimensional "
                                       "vector {}; likely a header".format(word, entries))
                        continue
                    elif dim != len(entries):
                        raise RuntimeError(
                            "Vector for token {} has {} dimensions, but previously "
                            "read vectors have {} dimensions. All vectors must have "
                            "the same number of dimensions.".format(word, len(entries),
                                                                    dim))

                    try:
                        if isinstance(word, bytes):
                            word = word.decode('utf-8')
                    except UnicodeDecodeError:
                        logger.info("Skipping non-UTF8 token {}".format(repr(word)))
                        continue

                    vectors[vectors_loaded] = torch.tensor([float(x) for x in entries])
                    vectors_loaded += 1
                    itos.append(word)

                    if vectors_loaded == max_vectors:
                        break

            self.itos = itos
            self.stoi = {word: i for i, word in enumerate(itos)}
            self.vectors = torch.Tensor(vectors).view(-1, dim)
            self.dim = dim
            logger.info('Saving vectors to {}'.format(path_pt))
            if not os.path.exists(cache):
                os.makedirs(cache)
            torch.save((self.itos, self.stoi, self.vectors, self.dim), path_pt)
        else:
            logger.info('Loading vectors from {}'.format(path_pt))
            self.itos, self.stoi, self.vectors, self.dim = torch.load(path_pt)

    def __len__(self):
        return len(self.vectors)

    def get_vecs_by_tokens(self, tokens, lower_case_backup=False):
        """Look up embedding vectors of tokens.

        Args:
            tokens: a token or a list of tokens. if `tokens` is a string,
                returns a 1-D tensor of shape `self.dim`; if `tokens` is a
                list of strings, returns a 2-D tensor of shape=(len(tokens),
                self.dim).
            lower_case_backup : Whether to look up the token in the lower case.
                If False, each token in the original case will be looked up;
                if True, each token in the original case will be looked up first,
                if not found in the keys of the property `stoi`, the token in the
                lower case will be looked up. Default: False.

        Examples:
            >>> examples = ['chip', 'baby', 'Beautiful']
            >>> vec = text.vocab.GloVe(name='6B', dim=50)
            >>> ret = vec.get_vecs_by_tokens(tokens, lower_case_backup=True)
        """
        to_reduce = False

        if not isinstance(tokens, list):
            tokens = [tokens]
            to_reduce = True

        if not lower_case_backup:
            indices = [self[token] for token in tokens]
        else:
            indices = [self[token] if token in self.stoi
                       else self[token.lower()]
                       for token in tokens]

        vecs = torch.stack(indices)
        return vecs[0] if to_reduce else vecs


class GloVe(Vectors):
    url = {
        '42B': 'http://nlp.stanford.edu/data/glove.42B.300d.zip',
        '840B': 'http://nlp.stanford.edu/data/glove.840B.300d.zip',
        'twitter.27B': 'http://nlp.stanford.edu/data/glove.twitter.27B.zip',
        '6B': 'http://nlp.stanford.edu/data/glove.6B.zip',
    }

    def __init__(self, name='840B', dim=300, **kwargs):
        url = self.url[name]
        name = 'glove.{}.{}d.txt'.format(name, str(dim))
        super(GloVe, self).__init__(name, url=url, **kwargs)


class FastText(Vectors):

    url_base = 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.{}.vec'

    def __init__(self, language="en", **kwargs):
        url = self.url_base.format(language)
        name = os.path.basename(url)
        super(FastText, self).__init__(name, url=url, **kwargs)


class CharNGram(Vectors):

    name = 'charNgram.txt'
    url = ('http://www.logos.t.u-tokyo.ac.jp/~hassy/publications/arxiv2016jmt/'
           'jmt_pre-trained_embeddings.tar.gz')

    def __init__(self, **kwargs):
        super(CharNGram, self).__init__(self.name, url=self.url, **kwargs)

    def __getitem__(self, token):
        vector = torch.Tensor(1, self.dim).zero_()
        if token == "<unk>":
            return self.unk_init(vector)
        chars = ['#BEGIN#'] + list(token) + ['#END#']
        num_vectors = 0
        for n in [2, 3, 4]:
            end = len(chars) - n + 1
            grams = [chars[i:(i + n)] for i in range(end)]
            for gram in grams:
                gram_key = '{}gram-{}'.format(n, ''.join(gram))
                if gram_key in self.stoi:
                    vector += self.vectors[self.stoi[gram_key]]
                    num_vectors += 1
        if num_vectors > 0:
            vector /= num_vectors
        else:
            vector = self.unk_init(vector)
        return vector


pretrained_aliases = {
    "charngram.100d": partial(CharNGram),
    "fasttext.en.300d": partial(FastText, language="en"),
    "fasttext.simple.300d": partial(FastText, language="simple"),
    "glove.42B.300d": partial(GloVe, name="42B", dim="300"),
    "glove.840B.300d": partial(GloVe, name="840B", dim="300"),
    "glove.twitter.27B.25d": partial(GloVe, name="twitter.27B", dim="25"),
    "glove.twitter.27B.50d": partial(GloVe, name="twitter.27B", dim="50"),
    "glove.twitter.27B.100d": partial(GloVe, name="twitter.27B", dim="100"),
    "glove.twitter.27B.200d": partial(GloVe, name="twitter.27B", dim="200"),
    "glove.6B.50d": partial(GloVe, name="6B", dim="50"),
    "glove.6B.100d": partial(GloVe, name="6B", dim="100"),
    "glove.6B.200d": partial(GloVe, name="6B", dim="200"),
    "glove.6B.300d": partial(GloVe, name="6B", dim="300")
}
"""Mapping from string name to factory function"""

'Mapping from string name to factory function'

In [ ]:
from collections import defaultdict
import logging
import torch
from tqdm import tqdm
from collections import Counter
# from torchtext.vocab import (
#     pretrained_aliases,  # not in legacy
#     Vectors,  # not in legacy
# )
from typing import List
logger = logging.getLogger(__name__)


class Vocab(object):
    """Defines a vocabulary object that will be used to numericalize a field.

    Attributes:
        freqs: A collections.Counter object holding the frequencies of tokens
            in the data used to build the Vocab.
        stoi: A collections.defaultdict instance mapping token strings to
            numerical identifiers.
        itos: A list of token strings indexed by their numerical identifiers.
    """

    # TODO (@mttk): Populate classs with default values of special symbols
    UNK = '<unk>'

    def __init__(self, counter, max_size=None, min_freq=1, specials=('<unk>', '<pad>'),
                 vectors=None, unk_init=None, vectors_cache=None, specials_first=True):
        """Create a Vocab object from a collections.Counter.

        Args:
            counter: collections.Counter object holding the frequencies of
                each value found in the data.
            max_size: The maximum size of the vocabulary, or None for no
                maximum. Default: None.
            min_freq: The minimum frequency needed to include a token in the
                vocabulary. Values less than 1 will be set to 1. Default: 1.
            specials: The list of special tokens (e.g., padding or eos) that
                will be prepended to the vocabulary. Default: ['<unk'>, '<pad>']
            vectors: One of either the available pretrained vectors
                or custom pretrained vectors (see Vocab.load_vectors);
                or a list of aforementioned vectors
            unk_init (callback): by default, initialize out-of-vocabulary word vectors
                to zero vectors; can be any function that takes in a Tensor and
                returns a Tensor of the same size. Default: 'torch.zeros'
            vectors_cache: directory for cached vectors. Default: '.vector_cache'
            specials_first: Whether to add special tokens into the vocabulary at first.
                If it is False, they are added into the vocabulary at last.
                Default: True.
        """
        self.freqs = counter
        counter = counter.copy()
        min_freq = max(min_freq, 1)

        self.itos = list()
        self.unk_index = None
        if specials_first:
            self.itos = list(specials)
            # only extend max size if specials are prepended
            max_size = None if max_size is None else max_size + len(specials)

        # frequencies of special tokens are not counted when building vocabulary
        # in frequency order
        for tok in specials:
            if tok in counter:
                del counter[tok]

        # sort by frequency, then alphabetically
        words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
        words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)

        for word, freq in words_and_frequencies:
            if freq < min_freq or len(self.itos) == max_size:
                break
            self.itos.append(word)

        if Vocab.UNK in specials:  # hard-coded for now
            unk_index = specials.index(Vocab.UNK)  # position in list
            # account for ordering of specials, set variable
            self.unk_index = unk_index if specials_first else len(self.itos) + unk_index
            self.stoi = defaultdict(self._default_unk_index)
        else:
            self.stoi = defaultdict()

        if not specials_first:
            self.itos.extend(list(specials))

        # stoi is simply a reverse dict for itos
        self.stoi.update({tok: i for i, tok in enumerate(self.itos)})

        self.vectors = None
        if vectors is not None:
            self.load_vectors(vectors, unk_init=unk_init, cache=vectors_cache)
        else:
            assert unk_init is None and vectors_cache is None

    def forward(self, tokens: List[str]) -> List[int]:
        return self.lookup_indices(tokens)

    def set_default_index(self, idx):
        self.unk_index = idx

    def _default_unk_index(self):
        return self.unk_index

    def __getitem__(self, token):
        return self.stoi.get(token, self.stoi.get(Vocab.UNK))

    def __getstate__(self):
        # avoid picking defaultdict
        attrs = dict(self.__dict__)
        # cast to regular dict
        attrs['stoi'] = dict(self.stoi)
        return attrs

    def __setstate__(self, state):
        if state.get("unk_index", None) is None:
            stoi = defaultdict()
        else:
            stoi = defaultdict(self._default_unk_index)
        stoi.update(state['stoi'])
        state['stoi'] = stoi
        self.__dict__.update(state)

    def __eq__(self, other):
        if self.freqs != other.freqs:
            return False
        if self.stoi != other.stoi:
            return False
        if self.itos != other.itos:
            return False
        if self.vectors != other.vectors:
            return False
        return True

    def __len__(self):
        return len(self.itos)

    def lookup_indices(self, tokens):
        indices = [self.__getitem__(token) for token in tokens]
        return indices

    def extend(self, v, sort=False):
        words = sorted(v.itos) if sort else v.itos
        for w in words:
            if w not in self.stoi:
                self.itos.append(w)
                self.stoi[w] = len(self.itos) - 1

    def load_vectors(self, vectors, **kwargs):
        """
        Args:
            vectors: one of or a list containing instantiations of the
                GloVe, CharNGram, or Vectors classes. Alternatively, one
                of or a list of available pretrained vectors:

                charngram.100d
                fasttext.en.300d
                fasttext.simple.300d
                glove.42B.300d
                glove.840B.300d
                glove.twitter.27B.25d
                glove.twitter.27B.50d
                glove.twitter.27B.100d
                glove.twitter.27B.200d
                glove.6B.50d
                glove.6B.100d
                glove.6B.200d
                glove.6B.300d

            Remaining keyword arguments: Passed to the constructor of Vectors classes.
        """
        if not isinstance(vectors, list):
            vectors = [vectors]
        for idx, vector in enumerate(vectors):
            if isinstance(vector, str):
                # Convert the string pretrained vector identifier
                # to a Vectors object
                if vector not in pretrained_aliases:
                    raise ValueError("Got string input vector {}, but allowed pretrained vectors are {}".format(vector, list(pretrained_aliases.keys())))
                vectors[idx] = pretrained_aliases[vector](**kwargs)
            elif not isinstance(vector, Vectors):
                raise ValueError( "Got input vectors of type {}, expected str or Vectors object".format(type(vector)))
        tot_dim = sum(v.dim for v in vectors)
        self.vectors = torch.Tensor(len(self), tot_dim)
        for i, token in enumerate(self.itos):
            start_dim = 0
            for v in vectors:
                end_dim = start_dim + v.dim
                self.vectors[i][start_dim:end_dim] = v[token.strip()]
                start_dim = end_dim
            assert(start_dim == tot_dim)

    def set_vectors(self, stoi, vectors, dim, unk_init=torch.Tensor.zero_):
        """
        Set the vectors for the Vocab instance from a collection of Tensors.

        Args:
            stoi: A dictionary of string to the index of the associated vector
                in the `vectors` input argument.
            vectors: An indexed iterable (or other structure supporting __getitem__) that
                given an input index, returns a FloatTensor representing the vector
                for the token associated with the index. For example,
                vector[stoi["string"]] should return the vector for "string".
            dim: The dimensionality of the vectors.
            unk_init (callback): by default, initialize out-of-vocabulary word vectors
                to zero vectors; can be any function that takes in a Tensor and
                returns a Tensor of the same size. Default: 'torch.zeros'
        """
        self.vectors = torch.Tensor(len(self), dim)
        for i, token in enumerate(self.itos):
            wv_index = stoi.get(token, None)
            if wv_index is not None:
                self.vectors[i] = vectors[wv_index]
            else:
                self.vectors[i] = unk_init(self.vectors[i])


class SubwordVocab(Vocab):

    def __init__(self, counter, max_size=None, specials=('<pad>'),
                 vectors=None, unk_init=torch.Tensor.zero_):
        """Create a revtok subword vocabulary from a collections.Counter.

        Args:
            counter: collections.Counter object holding the frequencies of
                each word found in the data.
            max_size: The maximum size of the subword vocabulary, or None for no
                maximum. Default: None.
            specials: The list of special tokens (e.g., padding or eos) that
                will be prepended to the vocabulary in addition to an <unk>
                token.
            vectors: One of either the available pretrained vectors
                or custom pretrained vectors (see Vocab.load_vectors);
                or a list of aforementioned vectors
            unk_init (callback): by default, initialize out-of-vocabulary word vectors
                to zero vectors; can be any function that takes in a Tensor and
                returns a Tensor of the same size. Default: 'torch.zeros
        """
        try:
            import revtok
        except ImportError:
            print("Please install revtok.")
            raise

        # Hardcode unk_index as subword_vocab has no specials_first argument
        self.unk_index = (specials.index(SubwordVocab.UNK)
                          if SubwordVocab.UNK in specials else None)

        if self.unk_index is None:
            self.stoi = defaultdict()
        else:
            self.stoi = defaultdict(self._default_unk_index)

        self.stoi.update({tok: i for i, tok in enumerate(specials)})
        self.itos = specials.copy()

        self.segment = revtok.SubwordSegmenter(counter, max_size)

        max_size = None if max_size is None else max_size + len(self.itos)

        # sort by frequency/entropy, then alphabetically
        toks = sorted(self.segment.vocab.items(), key=lambda tup: (len(tup[0]) != 1, -tup[1], tup[0]))

        for tok, _ in toks:
            if len(self.itos) == max_size:
                break
            self.itos.append(tok)
            self.stoi[tok] = len(self.itos) - 1

        if vectors is not None:
            self.load_vectors(vectors, unk_init=unk_init)


def build_vocab_from_iterator(iterator, num_lines=None):
    """
    Build a Vocab from an iterator.

    Args:
        iterator: Iterator used to build Vocab. Must yield list or iterator of tokens.
        num_lines: The expected number of elements returned by the iterator.
            (Default: None)
            Optionally, if known, the expected number of elements can be passed to
            this factory function for improved progress reporting.
    """

    counter = Counter()
    with tqdm(unit_scale=0, unit='lines', total=num_lines) as t:
        for tokens in iterator:
            counter.update(tokens)
            t.update(1)
    word_vocab = Vocab(counter)
    return word_vocab

# Tokenizer

In [ ]:
from collections import Counter, OrderedDict
from itertools import cycle
import re
import random
# from torchtext.vocab import vocab # Built it custom
from tqdm import tqdm
import warnings

class Tokenizer:
    """
    Tokenizer for processing symbolic mathematical expressions.
    """
    def __init__(self, df, index_token_pool_size, momentum_token_pool_size, special_symbols, UNK_IDX, to_replace):
        self.amps = df.Amplitude.tolist()
        self.sqamps = df['Squared Amplitude'].tolist()

        # Issue warnings if token pool sizes are too small
        if index_token_pool_size < 100:
            warnings.warn(f"Index token pool size ({index_token_pool_size}) is small. Consider increasing it.", UserWarning)
        if momentum_token_pool_size < 100:
            warnings.warn(f"Momentum token pool size ({momentum_token_pool_size}) is small. Consider increasing it.", UserWarning)

        # Generate token pools
        self.tokens_pool = [f"_{i}" for i in range(index_token_pool_size)]
        self.momentum_pool = [f"MOMENTUM_{i}" for i in range(momentum_token_pool_size)]

        # Regular expression patterns for token replacement
        self.pattern_momentum = re.compile(r'\b[p]_\d{1,}\b')
        self.pattern_num_123 = re.compile(r'\b(?![ps]_)\w+_\d{1,}\b')
        self.pattern_special = re.compile(r'\b\w+_+\w+\b\\')
        self.pattern_prop = re.compile(r'Prop')
        self.pattern_int = re.compile(r'int\{')
        self.pattern_operators = {
            '+': re.compile(r'\+'), '-': re.compile(r'-'), '*': re.compile(r'\*'),
            ',': re.compile(r','), '^': re.compile(r'\^'), '%': re.compile(r'%'),
            '}': re.compile(r'\}'), '(': re.compile(r'\('), ')': re.compile(r'\)')
        }
        self.function_opening = re.compile(r'(\w+_\{)')
        self.pattern_mass = re.compile(r'\b\w+_\w\b')
        self.pattern_s = re.compile(r'\b\w+_\d{2,}\b')
        self.pattern_reg_prop = re.compile(r'\b\w+_\d{1}\b')
        self.pattern_antipart = re.compile(r'(\w)_\w+_\d+\(X\)\^\(\*\)')
        self.pattern_part = re.compile(r'(\w)_\w+_\d+\(X\)')
        self.pattern_index = re.compile(r'\b\w+_\w+_\d{2,}\b')

        self.special_symbols = special_symbols
        self.UNK_IDX = UNK_IDX
        self.to_replace = to_replace

        self.bos_token_id = 0
        self.eos_token_id = 2
        self.pad_token_id = 1

    @staticmethod
    def remove_whitespace(expression):
        """Remove all forms of whitespace from the expression."""
        return re.sub(r'\s+', '', expression)

    @staticmethod
    def split_expression(expression):
        """Split the expression by space delimiter."""
        return re.split(r' ', expression)

    def build_tgt_vocab(self):
        """Build vocabulary for target sequences."""
        counter = Counter()
        for eqn in tqdm(self.sqamps, desc='Processing target vocab'):
            counter.update(self.tgt_tokenize(eqn))
        voc = Vocab(OrderedDict(counter), specials=self.special_symbols[:], specials_first=True)
        voc.set_default_index(self.UNK_IDX)
        return voc

    def build_src_vocab(self, seed):
        """Build vocabulary for source sequences."""
        counter = Counter()
        for diag in tqdm(self.amps, desc='Processing source vocab'):
            counter.update(self.src_tokenize(diag, seed))
        voc = Vocab(OrderedDict(counter), specials=self.special_symbols[:], specials_first=True)
        voc.set_default_index(self.UNK_IDX)
        return voc

    def src_replace(self, ampl, seed):
        """Replace indexed and momentum variables with tokenized equivalents."""
        ampl = self.remove_whitespace(ampl)

        random.seed(seed)
        token_cycle = cycle(random.sample(self.tokens_pool, len(self.tokens_pool)))
        momentum_cycle = cycle(random.sample(self.momentum_pool, len(self.momentum_pool)))

        # Replace momentum tokens
        temp_ampl = ampl
        momentum_mapping = {match: next(momentum_cycle) for match in set(self.pattern_momentum.findall(ampl))}
        for key, value in momentum_mapping.items():
            temp_ampl = temp_ampl.replace(key, value)


        def replace_123_match(match):
            word, num = match.group().rsplit('_', 1)
            # if word == 's':
            #     # Mandstein
            #     return match.group()
            # if word == 'p':
            #     # return match.group()  # Keep 'p_X' unchanged
            #     return f'MOMENTUM_{num}'
            # IDX_POOL.setdefault(num, len(IDX_POOL) + 1)
            return f"{word}_INDEX_{next(token_cycle)}"
        # Replace index tokens
        # num_123_mapping = {match: match.rsplit('_',1) + next(token_cycle) for match in set(self.pattern_num_123.findall(ampl))}
        # for key, value in num_123_mapping.items():
        #     temp_ampl = temp_ampl.replace(key, value)
        temp_ampl = re.sub(self.pattern_num_123,replace_123_match,ampl)
        return temp_ampl

    def src_tokenize(self, ampl, seed):
        """Tokenize source expression, optionally applying replacements."""
        temp_ampl = self.src_replace(ampl, seed) if self.to_replace else ampl
        temp_ampl = temp_ampl.replace('\\\\', '\\').replace('\\', ' \\ ').replace('%', '')

        for symbol, pattern in self.pattern_operators.items():
            temp_ampl = pattern.sub(f' {symbol} ', temp_ampl)

        temp_ampl = self.function_opening.sub(r'\1 ', temp_ampl)

        temp_ampl = re.sub(r' {2,}', ' ', temp_ampl)
        return [token for token in self.split_expression(temp_ampl) if token]

    def tgt_tokenize(self, sqampl):
        """Tokenize target expression."""
        sqampl = self.remove_whitespace(sqampl)
        temp_sqampl = sqampl

        for symbol, pattern in self.pattern_operators.items():
            temp_sqampl = pattern.sub(f' {symbol} ', temp_sqampl)

        for pattern in [self.pattern_reg_prop, self.pattern_mass, self.pattern_s]:
            temp_sqampl = pattern.sub(lambda match: f' {match.group(0)} ', temp_sqampl)

        temp_sqampl = re.sub(r' {2,}', ' ', temp_sqampl)
        return [token for token in self.split_expression(temp_sqampl) if token]

    def batch_decode(self, batch_of_tokens, vocab):
        """
        Decode a batch of token sequences back to their original text form.

        Parameters:
        -----------
        batch_of_tokens : List[List[int]]
            A batch of token sequences, where each sequence is a list of token indices
        vocab : Vocab
            The vocabulary object used for mapping indices to tokens

        Returns:
        --------
        List[str]
            The decoded expressions
        """
        decoded_expressions = []

        for token_sequence in batch_of_tokens:
            # Convert indices to tokens
            tokens = [vocab.itos[idx] for idx in token_sequence if idx != self.UNK_IDX]

            # # Replace special tokens with their original form
            # for i, token in enumerate(tokens):
            #     if token in self.token_to_original:
            #         tokens[i] = self.token_to_original[token]

            # Remove special symbols that might have been added
            tokens = [t for t in tokens if t not in self.special_symbols]

            # Join tokens into a string and remove excess spaces
            decoded_expr = ''.join(tokens)

            # Clean up the expression by removing spaces around operators
            for symbol in self.pattern_operators.keys():
                decoded_expr = decoded_expr.replace(f" {symbol} ", symbol)

            # # Further cleaning for specific patterns
            # decoded_expr = decoded_expr.replace('\\', '').replace(' ', '')

            decoded_expressions.append(decoded_expr)

        return decoded_expressions


In [ ]:
BOS_IDX, PAD_IDX, EOS_IDX, UNK_IDX, SEP_IDX = 0, 1, 2, 3, 4
special_symbols = ['<S>', '<PAD>', '</S>', '<UNK>', '<SEP>']
tokenizer = Tokenizer(train_df, 500, 500, special_symbols, UNK_IDX, False)

In [ ]:
def normalize_indices(tokenizer, expressions, index_token_pool_size=50, momentum_token_pool_size=50):
    # Function to replace indices with a new set of tokens for each expression
    def replace_indices(token_list, index_map):
        new_index = (f"_{i}" for i in range(index_token_pool_size))  # Local generator for new indices
        new_tokens = []
        for token in token_list:
            if "INDEX_" in token:
                if token not in index_map:
                    try:
                        index_map[token] = token.rsplit('_',1)[0] + next(new_index)
                    except StopIteration:
                        # Handle the case where no more indices are available
                        raise ValueError("Ran out of unique indices, increase token_pool_size")
                new_tokens.append(index_map[token])
            else:
                new_tokens.append(token)
        return new_tokens

    def replace_momenta(token_list, index_map):
        new_index = (f"MOMENTUM_{i}" for i in range(momentum_token_pool_size))  # Local generator for new indices
        new_tokens = []
        for token in token_list:
            if "MOMENTUM_" in token:
                if token not in index_map:
                    try:
                        index_map[token] = next(new_index)
                    except StopIteration:
                        # Handle the case where no more indices are available
                        raise ValueError("Ran out of unique indices, increase momentum_token_pool_size")
                new_tokens.append(index_map[token])
            else:
                new_tokens.append(token)
        return new_tokens

    normalized_expressions = []
    # Replace indices in each expression randomly
    for expr in tqdm(expressions,desc="Normalizing.."):
        toks = tokenizer.src_tokenize(expr,42)
        normalized_expressions.append(replace_momenta(replace_indices(toks, {}), {}))

    return normalized_expressions


def aug_data(df):
    # Extract columns
    amps = df['Amplitude']
    sqamps = df['Squared Amplitude']

    # Data augmentation
    n_samples = 1 #args.n_samples
    aug_amps = []

    for amp in tqdm(amps, desc='processing'):
        random_seed = [random.randint(1, 1000) for _ in range(n_samples)]
        for seed in random_seed:
            aug_amps.append(tokenizer.src_replace(amp, seed))
    aug_sqamps = [sqamp for sqamp in sqamps for _ in range(n_samples)]

    if True:
        normal_amps = normalize_indices(tokenizer, aug_amps, 500, 500)
        aug_amps = []
        for amp in normal_amps:
            aug_amps.append("".join(amp))

    # Create augmented DataFrame
    df_aug = pd.DataFrame({"Amplitude": aug_amps, "Squared Amplitude": aug_sqamps})

    return df_aug

In [ ]:
train_df_aug = aug_data(train_df)

Normalizing..: 100%|██████████| 12441/12441 [00:02<00:00, 5818.70it/s]


In [ ]:
val_df_aug = aug_data(val_df)

Normalizing..: 100%|██████████| 1555/1555 [00:00<00:00, 6083.72it/s]


In [ ]:
test_df_aug = aug_data(test_df)

Normalizing..: 100%|██████████| 1556/1556 [00:00<00:00, 5764.13it/s]


In [ ]:
# train_df_aug.to_csv('train_df_aug.csv')
# val_df_aug.to_csv('val_df_aug.csv')
# test_df_aug.to_csv('test_df_aug.csv')

In [ ]:
tokenizer2 = Tokenizer(train_df_aug, 500, 500, special_symbols, UNK_IDX, False)

In [ ]:
src_vocab2 = tokenizer2.build_src_vocab(42)
len(src_vocab2.itos)

Processing source vocab: 100%|██████████| 12441/12441 [00:01<00:00, 7834.75it/s]


459

In [ ]:
tgt_vocab2 = tokenizer2.build_tgt_vocab()
len(tgt_vocab2.itos)

Processing target vocab: 100%|██████████| 12441/12441 [00:01<00:00, 8034.43it/s]


59

In [ ]:
tokenizer2.special_symbols

['<S>', '<PAD>', '</S>', '<UNK>', '<SEP>']

In [ ]:
# tokenizer2.batch_decode([[2,10,21]],tgt_vocab2)

# dataset class

In [ ]:
#from fn_utils import causal_mask
from torch.utils.data import Dataset
import torch

#from ..constants import EOS_IDX,BOS_IDX,PAD_IDX
def causal_mask(size):
    """Create a causal mask for a sequence of given size."""
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).int()
    return mask == 0

class Data(Dataset):
    """
    Custom PyTorch dataset for handling data.

    Args:
        df (DataFrame): DataFrame containing data.
    """

    def __init__(self, df, tokenizer, config, src_vocab, tgt_vocab):
        super(Data, self).__init__()
        self.tgt_vals = df['Squared Amplitude']
        self.src_vals = df['Amplitude']
        self.tgt_tokenize = tokenizer.tgt_tokenize
        self.src_tokenize = tokenizer.src_tokenize
        self.bos_token = torch.tensor([BOS_IDX], dtype=torch.int64)
        self.eos_token = torch.tensor([EOS_IDX], dtype=torch.int64)
        self.pad_token = torch.tensor([PAD_IDX], dtype=torch.int64)
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.config = config

    def __len__(self):
        """
        Get the length of the dataset.

        Returns:
            int: Length of the dataset.
        """
        return len(self.src_vals)

    def __getitem__(self, idx):
        """
        Get an item from the dataset at the specified index.

        Args:
            idx (int): Index of the item.

        Returns:
            tuple: Tuple containing source and target tensors.
        """
        # print(f'index: {idx}')
        src_tokenized = self.src_tokenize(self.src_vals[idx],self.config.seed)
        tgt_tokenized = self.tgt_tokenize(self.tgt_vals[idx])
        src_ids = self.src_vocab.forward(src_tokenized)
        tgt_ids = self.tgt_vocab.forward(tgt_tokenized)

        enc_num_padding_tokens = self.config.src_max_len - len(src_ids) - 2
        dec_num_padding_tokens = self.config.tgt_max_len - len(tgt_ids) - 1
        # print(f'src_ids: {len(src_ids)} tgt_ids:  {len(tgt_ids)} enc_num: {enc_num_padding_tokens} dec_num: {dec_num_padding_tokens} \n' )
        if self.config.truncate:
            if enc_num_padding_tokens < 0:
                src_ids = src_ids[:self.config.src_max_len-2]
            if dec_num_padding_tokens < 0:
                tgt_ids = tgt_ids[:self.config.tgt_max_len-1]
        else:
            if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
                raise ValueError("Sentence is too long")
        src_tensor = torch.cat(
            [
                self.bos_token,
                torch.tensor(src_ids, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] *
                             enc_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        tgt_tensor = torch.cat(
            [
                self.bos_token,
                torch.tensor(tgt_ids, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] *
                             dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        label = torch.cat(
            [
                torch.tensor(tgt_ids, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        src_mask = (src_tensor != self.pad_token).unsqueeze(0).unsqueeze(0).int() # (1, 1, seq_len)
        tgt_mask = (tgt_tensor != self.pad_token).unsqueeze(0).int() & causal_mask(tgt_tensor.size(0)) # (1, seq_len) & (1, seq_len, seq_len),

        return src_tensor, tgt_tensor, label, src_mask, tgt_mask#, len(src_ids), len(tgt_ids)
        # return {'input_ids':src_tensor,'labels':label,'src_attn_mask':src_mask,'tgt_attn_mask':tgt_mask}

    @staticmethod
    def get_data(df_train, df_test, df_valid, config, tokenizer, src_vocab,tgt_vocab):
        """
        Create datasets (train, test, and valid)

        Returns:
            dict: Dictionary containing train, test, and valid datasets.
        """
        train = Data(df_train, tokenizer, config,src_vocab,tgt_vocab)
        test = Data(df_test, tokenizer, config,src_vocab,tgt_vocab)
        valid = Data(df_valid, tokenizer, config,src_vocab,tgt_vocab)

        return {'train': train, 'test': test, 'valid': valid}

In [ ]:
class d_config:
    src_max_len = 300
    tgt_max_len = 350
    truncate = False
    seed = 42


In [ ]:
dataset = Data.get_data(train_df_aug, test_df_aug, val_df_aug, d_config, tokenizer2, src_vocab2, tgt_vocab2)

# Helpers

## utils.py

## flash_cross_attention.py

In [ ]:
from typing import Optional
from functools import partial

import torch
import torch.nn as nn

from mamba_ssm.utils.generation import InferenceParams
from flash_attn.modules.mha import MHA
from x_transformers.x_transformers import Attention
from x_transformers.x_transformers import RelativePositionBias as xRelativePositionBias


from math import sqrt

try:
    from mamba_ssm.ops.triton.layer_norm import RMSNorm, layer_norm_fn, rms_norm_fn
except ImportError:
    RMSNorm, layer_norm_fn, rms_norm_fn = None, None, None


class FlashCrossAttentionWrapper(nn.Module):

    def __init__(
        self,
        layer_idx=0,
        d_model: int = 512,
        n_head: int = 8,
        rms_norm: bool = True,
        norm_epsilon: float = 1e-5,
        residual_in_fp32=True,
        fused_add_norm=True,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__()

        self.layer_idx = layer_idx
        self.residual_in_fp32 = residual_in_fp32
        self.fused_add_norm = fused_add_norm

        self.attention = Attention(
            dim=d_model,
            dim_head=d_model // n_head,
            heads=n_head,
            causal=False,  # x attention should not be causal
            dropout=dropout,
            flash=True,
        )

        self.rel_pos = xRelativePositionBias(
            scale= (1 / d_model // n_head) ** 0.5,
            heads=n_head,
        )

        # self.attention = MHA(
        #     embed_dim=d_model,
        #     num_heads=n_head,
        #     cross_attn=True,
        #     # causal=True,
        #     dropout=dropout,
        #     layer_idx=layer_idx,
        #     # use_flash_attn=True, # not supported if cross_attn
        # )

        norm_cls = partial(
            nn.LayerNorm if not rms_norm else RMSNorm,
            eps=norm_epsilon,  # **factory_kwargs
        )

        self.norm = norm_cls(d_model)

    def forward(
        self,
        hidden_states: torch.Tensor,
        context: Optional[torch.Tensor] = None,
        context_mask: Optional[torch.Tensor] = None,
        residual: Optional[torch.Tensor] = None,
        mask: Optional[torch.Tensor] = None,
        inference_params: InferenceParams = None,
    ):
        r"""Pass the input through the encoder layer.

        Args:
            hidden_states: the sequence to the encoder layer (required).
            residual: hidden_states = Mixer(LN(residual))
        """

        if not self.fused_add_norm:
            residual = (
                (hidden_states + residual) if residual is not None else hidden_states
            )
            hidden_states = self.norm(residual.to(dtype=self.norm.weight.dtype))
            if self.residual_in_fp32:
                residual = residual.to(torch.float32)
        else:
            fused_add_norm_fn = (
                rms_norm_fn if isinstance(self.norm, RMSNorm) else layer_norm_fn
            )
            hidden_states, residual = fused_add_norm_fn(
                hidden_states,
                self.norm.weight,
                self.norm.bias,
                residual=residual,
                prenorm=True,
                residual_in_fp32=self.residual_in_fp32,
                eps=self.norm.eps,
            )

        cache = None

        if inference_params is not None and inference_params.seqlen_offset > 0:
            cache = inference_params.key_value_memory_dict[self.layer_idx]
            cache.seqlen_offset = inference_params.seqlen_offset

        hidden_states, cache = self.attention.forward(
            x=hidden_states,
            mask=mask,
            context=context,
            context_mask=context_mask,
            # rel_pos=self.rel_pos,
            return_intermediates=True,
            cache=cache,
        )

        if inference_params is not None:
            inference_params.key_value_memory_dict[self.layer_idx] = cache
            # hidden_states = (
            #     hidden_states[:, -1:, :]
            #     if inference_params.seqlen_offset > 0
            #     else hidden_states
            # )

        return hidden_states, residual

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return None

## cross_attention.py

In [ ]:
from typing import Optional
from functools import partial

import torch
import torch.nn as nn

from mamba_ssm.utils.generation import InferenceParams
from torchscale.architecture.config import DecoderConfig
from torchscale.component.multihead_attention import MultiheadAttention
from torchscale.component.relative_position_bias import RelativePositionBias

try:
    from mamba_ssm.ops.triton.layer_norm import RMSNorm, layer_norm_fn, rms_norm_fn
except ImportError:
    RMSNorm, layer_norm_fn, rms_norm_fn = None, None, None


class CrossAttentionWrapper(nn.Module):

    def __init__(
        self,
        layer_idx=0,
        d_model: int = 512,
        n_head: int = 8,
        rms_norm: bool = True,
        norm_epsilon: float = 1e-5,
        residual_in_fp32=True,
        fused_add_norm=True,
        attention_dropout=0,
        **kwargs,
    ):
        super().__init__()

        self.layer_idx = layer_idx
        self.residual_in_fp32 = residual_in_fp32
        self.fused_add_norm = fused_add_norm

        args = DecoderConfig(
            rel_pos_buckets=32, max_rel_pos=128, decoder_attention_heads=n_head
        )

        self.attention = MultiheadAttention(
            args,
            d_model,
            num_heads=n_head,
            dropout=attention_dropout,
            self_attention=False,
            encoder_decoder_attention=True,
            subln=args.subln,
        )

        self.cross_attn_relative_position = RelativePositionBias(
            num_buckets=args.rel_pos_buckets,
            max_distance=args.max_rel_pos,
            n_heads=args.decoder_attention_heads,
        )

        norm_cls = partial(
            nn.LayerNorm if not rms_norm else RMSNorm,
            eps=norm_epsilon,  # **factory_kwargs
        )

        self.norm = norm_cls(d_model)

    def forward(
        self,
        hidden_states: torch.Tensor,
        context: Optional[torch.Tensor] = None,
        context_mask: Optional[torch.Tensor] = None,
        residual: Optional[torch.Tensor] = None,
        mask: Optional[torch.Tensor] = None,
        inference_params: InferenceParams = None,
    ):
        r"""Pass the input through the encoder layer.

        Args:
            hidden_states: the sequence to the encoder layer (required).
            residual: hidden_states = Mixer(LN(residual))
        """
        if not self.fused_add_norm:
            residual = (
                (hidden_states + residual) if residual is not None else hidden_states
            )
            hidden_states = self.norm(residual.to(dtype=self.norm.weight.dtype))
            if self.residual_in_fp32:
                residual = residual.to(torch.float32)
        else:
            fused_add_norm_fn = (
                rms_norm_fn if isinstance(self.norm, RMSNorm) else layer_norm_fn
            )
            hidden_states, residual = fused_add_norm_fn(
                hidden_states,
                self.norm.weight,
                self.norm.bias,
                residual=residual,
                prenorm=True,
                residual_in_fp32=self.residual_in_fp32,
                eps=self.norm.eps,
            )

        bsz, slen, _ = hidden_states.shape

        incremental_state = None

        if inference_params is not None and inference_params.seqlen_offset == 0:
            slen = 1
            incremental_state = {}

        elif inference_params is not None and inference_params.seqlen_offset > 0:
            slen = inference_params.seqlen_offset
            incremental_state = inference_params.key_value_memory_dict[self.layer_idx]

        cross_attn_rel_pos = self.cross_attn_relative_position(
            batch_size=bsz,
            qlen=slen,
            klen=context.size(1),
        )

        if inference_params is not None and inference_params.seqlen_offset != 0:
            cross_attn_rel_pos = cross_attn_rel_pos[-1:, :, :]

        hidden_states, attn = self.attention.forward(
            query=hidden_states,
            key=context,
            value=context,
            key_padding_mask=context_mask,
            incremental_state=incremental_state,
            # rel_pos=cross_attn_rel_pos,  # none in the default torchscale config
        )

        if inference_params is not None and inference_params.seqlen_offset == 0:
            inference_params.key_value_memory_dict[self.layer_idx] = incremental_state

        return hidden_states, residual

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return None

## ffn.py

In [ ]:
from typing import Optional
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
from mamba_ssm.utils.generation import InferenceParams
from flash_attn.modules.mlp import GatedMlp

try:
    from mamba_ssm.ops.triton.layer_norm import RMSNorm, layer_norm_fn, rms_norm_fn
except ImportError:
    RMSNorm, layer_norm_fn, rms_norm_fn = None, None, None


class FeedForwardWrapper(nn.Module):

    def __init__(
        self,
        layer_idx=0,
        d_model: int = 512,
        n_head: int = 8,
        rms_norm: bool = True,
        norm_epsilon: float = 1e-5,
        residual_in_fp32=True,
        fused_add_norm=True,
        attention_dropout=0,
        **kwargs,
    ):
        super().__init__()

        self.layer_idx = layer_idx
        self.residual_in_fp32 = residual_in_fp32
        self.fused_add_norm = fused_add_norm

        norm_cls = partial(
            nn.LayerNorm if not rms_norm else RMSNorm,
            eps=norm_epsilon,  # **factory_kwargs
        )

        self.norm = norm_cls(d_model)

        self.mlp = GatedMlp(
            in_features=d_model, hidden_features=4 * d_model, activation=F.silu
        )

    def forward(
        self,
        hidden_states: torch.Tensor,
        context: Optional[torch.Tensor] = None,
        context_mask: Optional[torch.Tensor] = None,
        residual: Optional[torch.Tensor] = None,
        mask: Optional[torch.Tensor] = None,
        inference_params: InferenceParams = None,
    ):
        r"""Pass the input through the encoder layer.

        Args:
            hidden_states: the sequence to the encoder layer (required).
            residual: hidden_states = Mixer(LN(residual))
        """

        if not self.fused_add_norm:
            residual = (
                (hidden_states + residual) if residual is not None else hidden_states
            )
            hidden_states = self.norm(residual.to(dtype=self.norm.weight.dtype))
            if self.residual_in_fp32:
                residual = residual.to(torch.float32)
        else:
            fused_add_norm_fn = (
                rms_norm_fn if isinstance(self.norm, RMSNorm) else layer_norm_fn
            )
            hidden_states, residual = fused_add_norm_fn(
                hidden_states,
                self.norm.weight,
                self.norm.bias,
                residual=residual,
                prenorm=True,
                residual_in_fp32=self.residual_in_fp32,
                eps=self.norm.eps,
            )

        hidden_states = self.mlp.forward(
            x=hidden_states,
        )

        return hidden_states, residual

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return None

## mamba.py

In [ ]:
from torch import Tensor

from typing import Optional

import torch
import torch.nn as nn


from functools import partial
from mamba_ssm.models.config_mamba import MambaConfig
from mamba_ssm.models.mixer_seq_simple import _init_weights
from mamba_ssm.utils.generation import InferenceParams
from mamba_ssm.modules.mamba_simple import Mamba


try:
    from mamba_ssm.ops.triton.layer_norm import RMSNorm, layer_norm_fn, rms_norm_fn
except ImportError:
    RMSNorm, layer_norm_fn, rms_norm_fn = None, None, None


class Block(nn.Module):
    def __init__(
        self,
        dim,
        mixer_cls,
        norm_cls=nn.LayerNorm,
        fused_add_norm=False,
        residual_in_fp32=False,
        layer_idx=None,
    ):
        """
        Simple block wrapping a mixer class with LayerNorm/RMSNorm and residual connection"

        This Block has a slightly different structure compared to a regular
        prenorm Transformer block.
        The standard block is: LN -> MHA/MLP -> Add.
        [Ref: https://arxiv.org/abs/2002.04745]
        Here we have: Add -> LN -> Mixer, returning both
        the hidden_states (output of the mixer) and the residual.
        This is purely for performance reasons, as we can fuse add and LayerNorm.
        The residual needs to be provided (except for the very first block).
        """
        super().__init__()
        self.layer_idx = layer_idx

        self.residual_in_fp32 = residual_in_fp32
        self.fused_add_norm = fused_add_norm
        self.mixer = mixer_cls(dim)
        self.norm = norm_cls(dim)
        if self.fused_add_norm:
            assert RMSNorm is not None, "RMSNorm import fails"
            assert isinstance(
                self.norm, (nn.LayerNorm, RMSNorm)
            ), "Only LayerNorm and RMSNorm are supported for fused_add_norm"

    def forward(
        self,
        hidden_states: Tensor,
        context=None,
        context_mask=None,
        residual: Optional[Tensor] = None,
        mask: Optional[Tensor] = None,
        inference_params: Optional[InferenceParams] = None,
    ):
        r"""Pass the input through the encoder layer.

        Args:
            hidden_states: the sequence to the encoder layer (required).
            residual: hidden_states = Mixer(LN(residual))
        """

        if not self.fused_add_norm:
            residual = (
                (hidden_states + residual) if residual is not None else hidden_states
            )
            hidden_states = self.norm(residual.to(dtype=self.norm.weight.dtype))
            if self.residual_in_fp32:
                residual = residual.to(torch.float32)
        else:
            fused_add_norm_fn = (
                rms_norm_fn if isinstance(self.norm, RMSNorm) else layer_norm_fn
            )
            hidden_states, residual = fused_add_norm_fn(
                hidden_states,
                self.norm.weight,
                self.norm.bias,
                residual=residual,
                prenorm=True,
                residual_in_fp32=self.residual_in_fp32,
                eps=self.norm.eps,
            )

        if inference_params is not None and inference_params.seqlen_offset > 0:
            hidden_states = hidden_states[:, -1:, :]

        hidden_states = self.mixer(
            hidden_states,
            # mask=mask,
            inference_params=inference_params
        )

        return hidden_states, residual

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return self.mixer.allocate_inference_cache(
            batch_size, max_seqlen, dtype=dtype, **kwargs
        )


def create_mamba_block(
    d_model,
    ssm_cfg=None,
    norm_epsilon=1e-5,
    rms_norm=False,
    residual_in_fp32=False,
    fused_add_norm=False,
    layer_idx=None,
    device=None,
    dtype=None,
    use_fast_path=True,
    dropout=0.0, # set in ssm_cfg
    **kwargs,
):
    if ssm_cfg is None:
        ssm_cfg = {}
    factory_kwargs = {"device": device, "dtype": dtype}
    mixer_cls = partial(
        Mamba,
        layer_idx=layer_idx,
        use_fast_path=use_fast_path,
        # **ssm_cfg,
        **factory_kwargs,
    )
    norm_cls = partial(
        nn.LayerNorm if not rms_norm else RMSNorm, eps=norm_epsilon, **factory_kwargs
    )
    block = Block(
        d_model,
        mixer_cls,
        norm_cls=norm_cls,
        fused_add_norm=fused_add_norm,
        residual_in_fp32=residual_in_fp32,
        layer_idx=layer_idx,
    )
    return block


class MixerModel(nn.Module):

    def __init__(
        self,
        d_model: int,
        n_layer: int,
        vocab_size: int,
        ssm_cfg=None,
        norm_epsilon: float = 1e-5,
        rms_norm: bool = False,
        initializer_cfg=None,
        fused_add_norm=False,
        residual_in_fp32=False,
        device=None,
        dtype=None,
        use_fast_path=True,
        layer_dict=None,
        **layer_kwargs,
    ) -> None:
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.residual_in_fp32 = residual_in_fp32

        self.embedding = nn.Embedding(vocab_size, d_model, **factory_kwargs)

        # We change the order of residual and layer norm:
        # Instead of LN -> Attn / MLP -> Add, we do:
        # Add -> LN -> Attn / MLP / Mixer, returning both the residual branch (output of Add) and
        # the main branch (output of MLP / Mixer). The model definition is unchanged.
        # This is for performance reason: we can fuse add + layer_norm.
        self.fused_add_norm = fused_add_norm
        if self.fused_add_norm:
            if layer_norm_fn is None or rms_norm_fn is None:
                raise ImportError("Failed to import Triton LayerNorm / RMSNorm kernels")

        layer_kwargs = {
            "d_model": d_model,
            "ssm_cfg": ssm_cfg,
            "norm_epsilon": norm_epsilon,
            "rms_norm": rms_norm,
            "residual_in_fp32": residual_in_fp32,
            "fused_add_norm": fused_add_norm,
            "use_fast_path": use_fast_path,
            "dropout": ssm_cfg["dropout"],
            **layer_kwargs,
            **factory_kwargs,
        }

        self.layers = nn.ModuleList(
            [
                (
                    create_mamba_block(
                        layer_idx=i,
                        **layer_kwargs,
                    )
                    if i not in layer_dict
                    else layer_dict[i](layer_idx=i, **layer_kwargs)  # access class
                )
                for i in range(n_layer)
            ]
        )

        self.norm_f = (nn.LayerNorm if not rms_norm else RMSNorm)(
            d_model, eps=norm_epsilon, **factory_kwargs
        )

        self.apply(
            partial(
                _init_weights,
                n_layer=n_layer,
                **(initializer_cfg if initializer_cfg is not None else {}),
            )
        )

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return {
            i: layer.allocate_inference_cache(
                batch_size, max_seqlen, dtype=dtype, **kwargs
            )
            for i, layer in enumerate(self.layers)
        }

    def forward(
        self,
        input_ids,
        context=None,
        context_mask=None,
        mask=None,
        inference_params: InferenceParams = None,
    ):
        hidden_states = self.embedding.forward(input_ids)
        residual = None
        for idx, layer in enumerate(self.layers):
            # print('layer: ',idx)
            hidden_states, residual = layer(
                hidden_states,
                context=context,
                context_mask=context_mask,
                residual=residual,
                mask=mask,
                inference_params=inference_params,
            )
        if not self.fused_add_norm:
            residual = (
                (hidden_states + residual) if residual is not None else hidden_states
            )
            hidden_states = self.norm_f(residual.to(dtype=self.norm_f.weight.dtype))
        else:
            # Set prenorm=False here since we don't need the residual
            fused_add_norm_fn = (
                rms_norm_fn if isinstance(self.norm_f, RMSNorm) else layer_norm_fn
            )
            hidden_states = fused_add_norm_fn(
                hidden_states,
                self.norm_f.weight,
                self.norm_f.bias,
                eps=self.norm_f.eps,
                residual=residual,
                prenorm=False,
                residual_in_fp32=self.residual_in_fp32,
            )
        return hidden_states


class MambaDecoder(
    nn.Module,
):

    def __init__(
        self,
        config: MambaConfig,
        initializer_cfg=None,
        device=None,
        dtype=None,
        layer_dict=None,
        layer_kwargs={},
    ) -> None:
        self.config = config
        d_model = config.d_model
        n_layer = config.n_layer
        vocab_size = config.vocab_size
        ssm_cfg = config.ssm_cfg
        rms_norm = config.rms_norm
        residual_in_fp32 = config.residual_in_fp32
        fused_add_norm = config.fused_add_norm
        pad_vocab_size_multiple = config.pad_vocab_size_multiple
        factory_kwargs = {"device": device, "dtype": dtype}

        super().__init__()
        # print(vocab_size)
        # if vocab_size % pad_vocab_size_multiple != 0:
        #     vocab_size += pad_vocab_size_multiple - (
        #         vocab_size % pad_vocab_size_multiple
        #     )
        # print(vocab_size)

        self.backbone = MixerModel(
            d_model=d_model,
            n_layer=n_layer,
            vocab_size=vocab_size,
            ssm_cfg=ssm_cfg,
            rms_norm=rms_norm,
            initializer_cfg=initializer_cfg,
            fused_add_norm=fused_add_norm,
            residual_in_fp32=residual_in_fp32,
            # use_fast_path=config.use_fast_path,
            layer_dict=layer_dict,
            **factory_kwargs,
            **layer_kwargs,
        )
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False, **factory_kwargs)

        # Initialize weights and apply final processing
        self.apply(
            partial(
                _init_weights,
                n_layer=n_layer,
                **(initializer_cfg if initializer_cfg is not None else {}),
            )
        )
        self.tie_weights()

    def tie_weights(self):
        self.lm_head.weight = self.backbone.embedding.weight

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return self.backbone.allocate_inference_cache(
            batch_size, max_seqlen, dtype=dtype, **kwargs
        )

    def forward(
        self,
        input_ids,
        context=None,
        context_mask=None,
        attention_mask=None,
        position_ids=None,
        inference_params=None,
        num_last_tokens=0,
    ):
        """
        "position_ids" is just to be compatible with Transformer generation. We don't use it.
        num_last_tokens: if > 0, only return the logits for the last n tokens
        """
        hidden_states = self.backbone(
            input_ids,
            context=context,
            context_mask=context_mask,
            mask=attention_mask,
            inference_params=inference_params,
        )
        if num_last_tokens > 0:
            hidden_states = hidden_states[:, -num_last_tokens:]
        return self.lm_head(hidden_states)

# mamba_encdec

In [ ]:
from typing import Any
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

from mamba_ssm.models.config_mamba import MambaConfig
from mamba_ssm.utils.generation import InferenceParams


# from utils.beam_search import BeamSearch
import json
# from utils.mt.comet import load_comet
from transformers.optimization import get_inverse_sqrt_schedule
from transformers import PreTrainedTokenizerFast
import evaluate

# from models.hybrids.helpers.flash_cross_attention import FlashCrossAttentionWrapper
# from models.hybrids.helpers.cross_attention import CrossAttentionWrapper
# from models.hybrids.helpers.ffn import FeedForwardWrapper
# from models.hybrids.helpers.mamba import MambaDecoder, MixerModel


class MambaEncDec(pl.LightningModule):
    is_encoder_decoder = True
    is_concat = False  # FIXME remove
    model_name = "mamba_encdec"
    configs = {
        "default": {
            "enc_n_layer": 5,
            # mamba config
            "d_model": 512,
            "n_layer": 5,
            "rms_norm": True,
            "fused_add_norm": True,
            "use_fast_path": False,
            "learning_rate": 1e-3,
            "warmup_steps": 500,
            "weight_decay": 0.01,
            "devices": 'cuda:0'
        }
    }

    def __init__(
        self,
        config=None,
        tokenizer=PreTrainedTokenizerFast,
        src_vocab_size=459,
        tgt_vocab_size=59,
        d_model=None,
        n_layer=None,
        enc_n_layer=None,
        rms_norm=None,
        fused_add_norm=None,
        use_fast_path=None,
        dropout=None,
        use_padding=None,
        precision="32-true",
        test_per_sample=True,
        test=False,
        test_suffix="",
        **kwargs,
    ):
        super().__init__()

        self.config = MambaConfig(
            vocab_size=tgt_vocab_size,
            d_model=d_model,
            n_layer=n_layer,
            rms_norm=rms_norm,
            fused_add_norm=fused_add_norm,
            # use_fast_path=use_fast_path,
            ssm_cfg={"dropout": dropout},
        )

        self.encoder = MixerModel(
            vocab_size=src_vocab_size,
            d_model=d_model,
            n_layer=enc_n_layer,
            rms_norm=rms_norm,
            fused_add_norm=fused_add_norm,
            use_fast_path=use_fast_path,
            ssm_cfg={"dropout": dropout},
            layer_dict={},
        )

        self.layers = (0, 3, 6, 9, 12, 15)
        x_attention_layers = [
            (i, FlashCrossAttentionWrapper) for i in (1, 4, 7, 10, 13, 16)
        ]
        ffn_layers = [(i, FeedForwardWrapper) for i in (2, 5, 8, 11, 14, 17)]

        layer_dict = dict(x_attention_layers + ffn_layers)

        self.decoder = MambaDecoder(
            config=self.config,
            layer_dict=layer_dict,
            layer_kwargs={"dropout":0.1}
        )

        self.tokenizer = tokenizer
        self.bleu = evaluate.load("sacrebleu")
        self.config = config
        self.use_padding = use_padding
        dtype_map = {
            "bf16-mixed": torch.bfloat16,
            "16-true": torch.float16,
            "32-true": torch.float32,
        }
        self.precision = dtype_map[precision]

        if test:
            # self.comet = load_comet()
            self.test_per_sample = test_per_sample
            self.test_res = []
            self.test_suffix = test_suffix

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return self.decoder.allocate_inference_cache(
            batch_size, max_seqlen, dtype=dtype, **kwargs
        )

    def forward(
        self,
        context_tokens,
        input_ids,
        source_attention_mask=None,
        target_attention_mask=None,
        position_ids=None,
        inference_params=None,
        num_last_tokens=0,
    ):

        b, _, _, l = source_attention_mask.shape
        source_attention_mask = source_attention_mask.reshape(b,l).to(torch.bool)
        target_attention_mask = target_attention_mask.to(torch.bool)

        source_vec = self.encoder.forward(
            input_ids=context_tokens,
            mask=source_attention_mask,
        )
        # print(source_vec.dtype, source_attention_mask.dtype)
        cache = self.allocate_inference_cache(
            batch_size=b,
            max_seqlen=300 + l + 1,  # source + BOS
            dtype=self.precision,
        )
        inference_params = InferenceParams(
            max_seqlen=300 + l + 1,
            max_batch_size=b,
            key_value_memory_dict=cache,
        )

        
        logits = self.decoder.forward(
            input_ids,
            context=source_vec,
            context_mask=source_attention_mask,
            attention_mask=target_attention_mask,
            position_ids=position_ids,
            inference_params=inference_params,
            num_last_tokens=num_last_tokens,
        )
        return logits

    def training_step(self, batch, batch_idx):
        # source, target, source_attention_mask = (
        #     batch["input_ids"],
        #     batch["labels"],
        #     batch["src_attention_mask"],
        # )
        source, target, _, source_attention_mask, _ = batch
        # source, target, source_attention_mask,

        target_attention_mask = (
            (target != self.tokenizer.pad_token_id).to(torch.bool).to(source.device)
        )
        # print(source.type(), source_attention_mask.type())

        lm_logits = self.forward(
            context_tokens=source,
            source_attention_mask=source_attention_mask,
            target_attention_mask=target_attention_mask,
            input_ids=target,
        )

        logits = lm_logits[:, :-1].contiguous()
        labels = target[:, 1:].contiguous()

        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            labels.view(-1),
            ignore_index=self.tokenizer.pad_token_id,
        )
        # self.log("train_loss", loss, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # src_tokens, labels, source_attention_mask = (
        #     batch["input_ids"],
        #     batch["labels"],
        #     batch["src_attention_mask"],
        # )
        src_tokens, target, _, source_attention_mask, _ = batch

        batch_size, seq_len = src_tokens.shape
        max_length = 351

        target_attention_mask = (
            (target != self.tokenizer.pad_token_id).to(torch.bool).to(src_tokens.device)
        )

        lm_logits = self.forward(
            context_tokens=src_tokens,
            source_attention_mask=source_attention_mask,
            target_attention_mask=target_attention_mask,
            input_ids=target,
        )

        logits = lm_logits[:, :-1].contiguous()
        labels = target[:, 1:].contiguous()

        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            labels.view(-1),
            ignore_index=self.tokenizer.pad_token_id,
        )

        return loss

    def test_step(self, batch, batch_idx):
        """beam search with parallel formulation"""
        # num_beams = 1

        # # source_tokens, labels, source_attention_mask = (
        # #     batch["input_ids"],
        # #     batch["labels"],
        # #     batch["attention_mask"],
        # # )
        src_tokens, _, labels, source_attention_mask, _ = batch
        batch_size, seq_len = src_tokens.shape
        max_length = 350
        cache = self.allocate_inference_cache(
            batch_size=batch_size,
            max_seqlen=max_length + seq_len + 1,  # source + BOS
            dtype=self.precision,
            # dtype = ,
        )
        inference_params = InferenceParams(
            max_seqlen=max_length + seq_len + 1,
            max_batch_size=batch_size,
            key_value_memory_dict=cache,
        )

        done = torch.tensor([False] * batch_size).to(src_tokens.device)
        preds = (
            torch.ones((batch_size, 1), dtype=torch.long).to(src_tokens.device)
            * self.tokenizer.bos_token_id
        )

        source_vec = self.encoder.forward(
            input_ids=src_tokens,
            mask=source_attention_mask,
        )
        # print('Source vec:', source_vec)
        position_ids = None

        for idx in range(labels.size(1)):

            if idx > 0:
                last_tokens = preds[:, -1:]  # (B, 1)
                position_ids = torch.full(
                    (batch_size, 1),
                    inference_params.seqlen_offset,
                    dtype=torch.long,
                    device=src_tokens.device,
                )
            
            logits = self.decoder.forward(
                input_ids=preds if idx == 0 else last_tokens,
                context=source_vec,
                position_ids=position_ids,
                inference_params=inference_params,
                num_last_tokens=1,
            )

            next_token_logits = logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            preds = torch.cat((preds, next_token), dim=-1)
            inference_params.seqlen_offset += 1
            # print(next_token.dtype)
            is_eos = next_token == self.tokenizer.eos_token_id
            done = done | is_eos.squeeze(-1)

            if done.all():
                break

        # Create a cumulative sum mask where positions after EOS become True
        eos_token_id = self.tokenizer.eos_token_id
        eos_mask = (preds == eos_token_id).cumsum(dim=1) > 0
        preds[eos_mask] = self.tokenizer.pad_token_id

        preds = preds.cpu()
        labels = labels.cpu()

        import gc
        
        del cache, logits, next_token_logits, inference_params

        if position_ids is not None:
            position_ids = position_ids.cpu()
            del position_ids
            # gc.collect()
        eos_mask = eos_mask.cpu()
        del eos_mask
        gc.collect()

        torch.cuda.empty_cache()
        
        return preds, labels #(token_acc, seq_acc)
        
    def on_test_epoch_end(self):
        # if self.test_per_sample:
        if False:
            source, target = self.config["language_pair"]

            with open(
                f"mt/res/{self.config['dataset']}/{self.config['dataset']}-{source}-{target}-{self.model_name}-{self.test_suffix}.json",
                "w",
            ) as f:
                json.dump(self.test_res, f)

    def _reorder_cache(self, cache, beam_idx):
        for layer_idx in self.layers:
            device = cache[layer_idx][0].device
            # {0:(conv_state, ssm_state)}
            cache[layer_idx] = (
                cache[layer_idx][0].index_select(0, beam_idx.to(device)),
                cache[layer_idx][1].index_select(0, beam_idx.to(device)),
            )
        return cache

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.config["learning_rate"],
            weight_decay=self.config["weight_decay"],
            fused=True,
        )

        scheduler = {
            "scheduler": get_inverse_sqrt_schedule(
                optimizer,
                num_warmup_steps=self.config["warmup_steps"],
            ),
            "interval": "step",
        }

        return {"optimizer": optimizer, "lr_scheduler": scheduler}

In [ ]:
# # best_model_path = '/kaggle/input/ckpt-mamba-e10/model-epoch10-valloss0.00.pt'
# # best_model_path = '/kaggle/input/ssm-ep5-ckpt/model-epoch05-valloss0.01.pt'
# best_model_path = '/kaggle/input/e4-d9-ssm-ep18/model-epoch18-valloss0.00.pt'
model = MambaEncDec(**MambaEncDec.configs['default'],config = MambaEncDec.configs['default'], tokenizer = tokenizer2, src_vocab_size = len(src_vocab2.itos), tgt_vocab_size = len(tgt_vocab2.itos))
best_model_path = '/kaggle/input/ssm-best-ckpt/model-epoch18-valloss0.00 (1).pt'
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])

<ipython-input-44-24ab68e13f22>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(best_model_path)


<All keys matched successfully>

# Training

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import numpy as np

# Hyperparameters
batch_size = 16
learning_rate = 1e-3
weight_decay = 0.01
num_epochs = 30
grad_accumulation_steps = 8
max_grad_norm = 1.0
warmup_steps = 1000
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
from torch.utils.data.distributed import DistributedSampler
# Create dataloaders
dataloaders = {
    split: DataLoader(
        dataset[split],
        batch_size=batch_size if split != 'test' else 4,
        shuffle=(split == 'train'),
        pin_memory=True,
        num_workers=4,
        # sampler= DistributedSampler(dataset[split])
    ) for split in ['train', 'valid', 'test']
}

# Testing

In [ ]:
def calculate_accuracy(model, dataloader, device):
    model.eval()
    model.to(device)
    total_seq = 0
    correct_seq = 0
    total_tokens = 0
    correct_tokens = 0
    loss_sum = 0

    with torch.no_grad():
        for idx, batch in tqdm(enumerate(dataloader), desc="Calculating accuracy",total=len(dataloader)):
            if idx>40:
                break
            # Move batch to device
            batch = [x.to(device) for x in batch]
            outputs, labels = model.test_step(batch, 0)
            outputs = outputs.cpu()[:,1:]
            l = outputs.shape[1]
            labels = labels.cpu()
            labels = labels[:,:l]
            labels[labels == 2] = 1
            
            total_tokens += outputs.numel()
            correct_tokens += ((outputs == labels)).sum().item()
            
            batch_size = labels.size(0)
            seq_correct = torch.all((outputs == labels), dim=1)
            correct_seq += seq_correct.sum().item()
            total_seq += batch_size



    # Calculate metrics
    token_accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0
    sequence_accuracy = correct_seq / total_seq if total_seq > 0 else 0
    avg_loss = loss_sum / len(dataloader)

    return {
        'token_accuracy': token_accuracy,
        'sequence_accuracy': sequence_accuracy,
        'loss': avg_loss
    }

Only run a fraction of batches due to slow autoregressive generation of SSMs

In [ ]:
# out = calculate_accuracy(trained_model, dataloaders['test'], device='cuda:1')
out = calculate_accuracy(model, dataloaders['test'], device='cuda:1')
model = model.cpu()

Calculating accuracy:  11%|█         | 41/389 [08:11<1:09:32, 11.99s/it]


In [ ]:
out

{'token_accuracy': 0.946803900325027,
 'sequence_accuracy': 0.9146341463414634,
 'loss': 0.0}

#####